In [2]:
import json

with open('credentials.json') as f:
    data = json.load(f)
    cred = data["Credentials"]
    access_token = cred['ACCESS_TOKEN']
    access_token_secret = cred['ACCESS_TOKEN_SECRET']
    consumer_key = cred['CONSUMER_KEY']
    consumer_secret = cred['CONSUMER_SECRET']


In [ ]:
import sys

from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener

# Replace the "None"s by your own credentials
ACCESS_TOKEN = access_token
ACCESS_TOKEN_SECRET = access_token_secret
CONSUMER_KEY = consumer_key
CONSUMER_SECRET = consumer_secret

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)


class Listener(StreamListener):
    def __init__(self, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.output_file = output_file
    def on_status(self, status):
        location = status.user.location
        hashtags = status.entities['hashtags']
        
        if location != None and len(hashtags) > 0:
            print(json.dumps(status._json), file=self.output_file)
                
    def on_error(self, status_code):
        print(status_code)
        return False
    
    

output = open('stream_output_user_location.json', 'w')
listener = Listener(output_file=output)

stream = Stream(auth=api.auth, listener=listener)
try:
    print('Start streaming.')
    stream.sample(languages=['en'])
except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()
    output.close()

Start streaming.
